<a href="https://colab.research.google.com/github/ollihansen90/Mathe-SH/blob/main/SudokuSolver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget -nc -q https://github.com/ollihansen90/Mathe-SH/raw/main/utils/utils_sudoku.py

In [ ]:
from utils_sudoku import erstelle_sudoku, plotte
from copy import deepcopy

In [ ]:
def initialisiere():
    eintraege = "123456789"
    liste = []
    for i in range(9):
        zeile = []
        for j in range(9):
            zeile.append(eintraege)
        liste.append(zeile)
    return liste

def einlesen(solver, sudoku_leer):
    s = deepcopy(solver)
    for i in range(9):
        for j in range(9):
            if sudoku_leer[i][j]!=" ":
                s[i][j] = sudoku_leer[i][j]
    return s

def plotte_solver(solver):
    s = deepcopy(solver)
    output = []
    for i in range(9):
        zeile = ""
        for j in range(9):
            eintrag = s[i][j]
            if len(eintrag)!=1:
                zeile = zeile+" "
            else:
                zeile = zeile+eintrag
        output.append(zeile)
    plotte(output)

def loesche_doppelte(solver):
    s = deepcopy(solver)
    for i in range(9):
        zeile = s[i]
        bekannte = ""
        for j in range(9):
            eintrag = zeile[j]
            if len(eintrag)==1:
                bekannte = bekannte+eintrag

        # Schritt 1: Finde Eintrag mit len(eintrag)>1
        for k in range(9):
            eintrag = zeile[k]
            if len(eintrag) != 1:

                # Schritt 2: Eliminiere alle "bekannte" aus "eintrag"
                for l in bekannte:
                    eintrag = eintrag.replace(str(l), "")

            # Schritt 3: Überschreibe Zeile s[i]
            s[i][k] = eintrag
    return s

def transponiere_solver(solver):
    s = deepcopy(solver)
    out = []
    for i in range(9):
        inner = []
        for j in range(9):
            inner.append(s[j][i])
        out.append(inner)
    return out

def loesche_spaltenweise(solver):
    s = deepcopy(solver)
    s = transponiere_solver(s)
    s = loesche_doppelte(s)
    s = transponiere_solver(s)
    return s

def loesche_3x3(solver):
    s = deepcopy(solver)
    bekannt = getblock(s)
    for i in range(9):
        for j in range(9):
            eintrag = s[i][j]
            if len(eintrag)>1:
                block_i = i//3
                block_j = j//3
                for zahl in bekannt[block_i][block_j]:
                    eintrag = eintrag.replace(zahl, "")
            s[i][j] = eintrag
    return s

def getblock(solver):
    s = deepcopy(solver)
    out = [["","",""],["","",""],["","",""]]
    for i in range(9):
        for j in range(9):
            eintrag = s[i][j]
            if len(eintrag)==1:
                block_i = i//3
                block_j = j//3
                out[block_i][block_j] = out[block_i][block_j]+eintrag
    return out

def loese(solver):
    s = deepcopy(solver)
    s_alt = None
    while s!=s_alt:
        s_alt = deepcopy(s)
        s = loesche_doppelte(s)
        s = loesche_spaltenweise(s)
        s = loesche_3x3(s)
    return s

def beseitige_zp(solver):
    s = deepcopy(solver)
    for i in range(9):
        zweier = []
        for j in range(9):
            if len(s[i][j])==2:
                zweier.append(s[i][j])
        if len(zweier)>1:
            loeschlist = []
            for h in range(0, len(zweier)-1):
                for k in range(h+1, len(zweier)):
                    if zweier[h]==zweier[k]:
                        loeschlist.append(zweier[h])
            if len(loeschlist)>0:
                for l in loeschlist:
                    for j in range(9):
                        if len(s[i][j])>1 and s[i][j]!=l:
                            s[i][j] = s[i][j].replace(l[0], "")
                            s[i][j] = s[i][j].replace(l[1], "")
    return s

def beseitige_zp_spalte(solver):
    s = deepcopy(solver)
    s = transponiere_solver(s)
    s = beseitige_zp(s)
    return transponiere_solver(s)

def beseitige_3x3_zp(solver):
    s = deepcopy(solver)
    s = block_zu_zeile(s)
    s = beseitige_zp(s)
    s = block_zu_zeile(s)
    return s

def block_zu_zeile(solver):
    s = deepcopy(solver)
    s_t = []

    for i in range(9):
        if i%3==0:
            s_t.append(s[i][:3])
            s_t.append(s[i][3:6])
            s_t.append(s[i][6:])
        else:
            if i%3==1:
                j = i
            else:
                j = i-1
            s_t[j-1].extend(s[i][:3])
            s_t[j].extend(s[i][3:6])
            s_t[j+1].extend(s[i][6:])
    return s_t

def loese(solver):
    s = deepcopy(solver)
    s_alt = None
    while s!=s_alt:
        s_alt = deepcopy(s)
        s = loesche_doppelte(s)
        s = loesche_spaltenweise(s)
        s = loesche_3x3(s)
        s = beseitige_zp(s)
        s = beseitige_zp_spalte(s)
        s = beseitige_3x3_zp(s)
    return s

In [ ]:
solver = initialisiere()
sudoku_leer = erstelle_sudoku(p=50)
#plotte(sudoku_leer)
solver = einlesen(solver, sudoku_leer)
plotte_solver(solver)

geloest = loese(solver)
plotte_solver(geloest)
print(geloest)

+---+---+---+---+---+---+---+---+---+
| 3   6     | 4       1 |           | 
+   +   +   +   +   +   +   +   +   +
|           | 7   3     |     8   6 | 
+   +   +   +   +   +   +   +   +   +
|           |         2 | 3   4   1 | 
+---+---+---+---+---+---+---+---+---+
|           |     1   7 |           | 
+   +   +   +   +   +   +   +   +   +
|         7 |           | 8       2 | 
+   +   +   +   +   +   +   +   +   +
| 2   5     |           | 7         | 
+---+---+---+---+---+---+---+---+---+
|     2     |     4     | 1         | 
+   +   +   +   +   +   +   +   +   +
|           |     7   5 | 9         | 
+   +   +   +   +   +   +   +   +   +
|     9   5 | 1         |     7   8 | 
+---+---+---+---+---+---+---+---+---+
+---+---+---+---+---+---+---+---+---+
| 3   6     | 4       1 |     9   7 | 
+   +   +   +   +   +   +   +   +   +
|           | 7   3   9 |     8   6 | 
+   +   +   +   +   +   +   +   +   +
|           |         2 | 3   4   1 | 
+---+---+---+---+---+---+---+---+---+


In [ ]:
plotte_solver(beseitige_zp(geloest))

['59']
['57']
['57']
[]
+---+---+---+---+---+---+---+---+---+
| 7   4   3 | 6   8   1 |     2     | 
+   +   +   +   +   +   +   +   +   +
| 8   2   6 | 5   9   7 | 3   4   1 | 
+   +   +   +   +   +   +   +   +   +
| 1   5   9 | 4   2   3 | 7   8   6 | 
+---+---+---+---+---+---+---+---+---+
| 2   1     | 3   4   9 | 6       8 | 
+   +   +   +   +   +   +   +   +   +
| 6   3     | 8   1   2 | 4   9     | 
+   +   +   +   +   +   +   +   +   +
| 9   8   4 | 7   5   6 | 1   3   2 | 
+---+---+---+---+---+---+---+---+---+
| 4   6   2 | 1   3   8 |           | 
+   +   +   +   +   +   +   +   +   +
| 5   7   8 | 9   6   4 | 2   1   3 | 
+   +   +   +   +   +   +   +   +   +
| 3   9   1 | 2   7   5 | 8   6   4 | 
+---+---+---+---+---+---+---+---+---+


In [ ]:
def beseitige_3x3_zp(solver):
    s = deepcopy(solver)
    s = block_zu_zeile(s)
    s = beseitige_zp(s)
    s = block_zu_zeile(s)
    return s


def block_zu_zeile(solver):
    s = deepcopy(solver)
    s_t = []

    for i in range(9):
        if i%3==0:
            s_t.append(s[i][:3])
            s_t.append(s[i][3:6])
            s_t.append(s[i][6:])
        else:
            if i%3==1:
                j = i
            else:
                j = i-1
            #print(j)
            s_t[j-1].extend(s[i][:3])
            s_t[j].extend(s[i][3:6])
            s_t[j+1].extend(s[i][6:])
    return s_t

plotte_solver(geloest)
plotte_solver(block_zu_zeile(geloest))

+---+---+---+---+---+---+---+---+---+
| 4   8   1 | 7   2   5 | 3   9   6 | 
+   +   +   +   +   +   +   +   +   +
| 7   3   2 | 6   9   8 | 1   4   5 | 
+   +   +   +   +   +   +   +   +   +
| 6   5   9 | 3   1   4 | 2   8   7 | 
+---+---+---+---+---+---+---+---+---+
| 5   4   3 | 2   6   9 | 7   1   8 | 
+   +   +   +   +   +   +   +   +   +
| 9   7   8 | 1   4   3 | 6   5   2 | 
+   +   +   +   +   +   +   +   +   +
| 2   1   6 | 8   5   7 | 9   3   4 | 
+---+---+---+---+---+---+---+---+---+
| 3   2   4 | 5   7   1 | 8   6   9 | 
+   +   +   +   +   +   +   +   +   +
| 8   9   7 | 4   3   6 | 5   2   1 | 
+   +   +   +   +   +   +   +   +   +
| 1   6   5 | 9   8   2 | 4   7   3 | 
+---+---+---+---+---+---+---+---+---+
+---+---+---+---+---+---+---+---+---+
| 4   8   1 | 7   3   2 | 6   5   9 | 
+   +   +   +   +   +   +   +   +   +
| 7   2   5 | 6   9   8 | 3   1   4 | 
+   +   +   +   +   +   +   +   +   +
| 3   9   6 | 1   4   5 | 2   8   7 | 
+---+---+---+---+---+---+---+---+---+
